In [20]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
from datetime import date, timedelta,datetime
import psycopg2
from sqlalchemy import create_engine
import sqlconfig
from sqlalchemy.exc import SQLAlchemyError

In [ ]:
#Finds all games in the 2024-25 season
def games():
    try:
        result = leaguegamefinder.LeagueGameFinder()
        all_games = result.get_data_frames()[0]
    except requests.exceptions.Timeout:
        print("Time out Error")
        all_games=[]

    #Dates to extract from all games from this year
    today_date=date.today()
    season_start_data = "2024-10-22"

    season_start_data = datetime.strptime(season_start_data, "%Y-%m-%d").date()

    #Data Wrangling to clean form
    all_games['GAME_DATE']=pd.to_datetime(all_games['GAME_DATE']).dt.date
    teams=['CLE','BOS','NYK','MIL','IND','DET','ORL','MIA','ATL','CHI','PHI','BRK','TOR','CHO','WAS','OKC','HOU','MEM','DEN','LAL','LAC','MIN','PHO','DAL','SAC','GSW','SAS','POR','NOP','UTA']
    games_2024_2025=all_games.loc[(all_games['GAME_DATE']>=season_start_data) & (all_games['GAME_DATE']<=today_date)]
    games_2024_2025=games_2024_2025[games_2024_2025['TEAM_ABBREVIATION'].isin(teams)]
    games_2024_2025=games_2024_2025.sort_values(by=['GAME_DATE','GAME_ID'],ascending=False)
    print(games_2024_2025.head(5))

    #Extracting columns for BCNF

    #NBA_GAME_TEAM_STATS
    NBA_game_team_stats=games_2024_2025[["GAME_ID","TEAM_ID","WL","MIN","FGM","FGA","FG3M","FG3A","FTM","FTA","OREB","DREB","AST","STL","BLK","PF","PLUS_MINUS"]]
    NBA_game_team_stats.columns=['game_id','team_id','wl','min','fgm','fga','fg3m','fg3a','ftm','fta','oreb','dreb','ast','stl','blk','pf','plus_minus']
    print("NBA_game_team_stats")
    print(NBA_game_team_stats.head(5))

    #NBA_Games
    nba_games=games_2024_2025[["GAME_ID","GAME_DATE"]].drop_duplicates()
    nba_games.columns=['game_id','game_date']
    return NBA_game_team_stats,nba_games

In [ ]:
def intitalInsertion(data,tablename):
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )
        
        # Insertions into the tables
        data.to_sql(tablename, engine, schema="nba24_25", if_exists="append", index=False)
      
        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
NBA_game_team_stats,nba_games=games()
intitalInsertion(nba_games,"nba_games")
intitalInsertion(NBA_game_team_stats,"nba_games_team_stats")

   SEASON_ID     TEAM_ID TEAM_ABBREVIATION              TEAM_NAME     GAME_ID  \
19     22024  1610612747               LAL     Los Angeles Lakers  0022400835   
24     22024  1610612742               DAL       Dallas Mavericks  0022400835   
27     22024  1610612744               GSW  Golden State Warriors  0022400834   
26     22024  1610612740               NOP   New Orleans Pelicans  0022400833   
31     22024  1610612759               SAS      San Antonio Spurs  0022400833   

     GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
19  2025-02-25  LAL vs. DAL  W  239  107  ...   0.750    13    44   57   24   
24  2025-02-25    DAL @ LAL  L  240   99  ...   0.923    14    31   45   23   
27  2025-02-25  GSW vs. CHA  W  241  128  ...   0.833    16    35   51   40   
26  2025-02-25  NOP vs. SAS  W  240  109  ...   0.800    20    34   54   25   
31  2025-02-25    SAS @ NOP  L  240  103  ...   0.778    13    32   45   29   

    STL  BLK  TOV  PF  PLUS_MINUS  
19